In [1]:
import os
import pandas as pd
from glob import glob
import tifffile as tiff
import json
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
import cv2
from PIL import Image, ImageOps
from tqdm.auto import tqdm
import numpy as np
import random

In [2]:
DATA_DIR = "D:Dataset/dataset/hubmap-organ-segmentation/"

# Training Datasets
TRAIN_CSV = os.path.join(DATA_DIR, 'train.csv')
train_df = pd.read_csv(TRAIN_CSV)
TRAIN_IMAGES_DIR = os.path.join(DATA_DIR, 'train_images')
TRAIN_LABELS_DIR = os.path.join(DATA_DIR, 'train_annotations')

all_train_images = glob(os.path.join(TRAIN_IMAGES_DIR, '*.tiff'), recursive = True)
all_train_labels = glob(os.path.join(TRAIN_LABELS_DIR, '*.json'), recursive = True)

print(train_df.shape)
display(train_df.head())

# Test Datasets
TEST_CSV = os.path.join(DATA_DIR, 'test.csv')
test_df = pd.read_csv(TEST_CSV)
TEST_IMAGES_DIR = os.path.join(DATA_DIR, 'test_images')

all_test_images = glob(os.path.join(TEST_IMAGES_DIR, '*.tiff'), recursive = True)

print(test_df.shape)
display(test_df)

(351, 10)


,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness,rle,age,sex
0,10044,prostate,HPA,3000,3000,0.4,4,1459676 77 1462675 82 1465674 87 1468673 92 14...,37.0,Male
1,10274,prostate,HPA,3000,3000,0.4,4,715707 2 718705 8 721703 11 724701 18 727692 3...,76.0,Male
2,10392,spleen,HPA,3000,3000,0.4,4,1228631 20 1231629 24 1234624 40 1237623 47 12...,82.0,Male
3,10488,lung,HPA,3000,3000,0.4,4,3446519 15 3449517 17 3452514 20 3455510 24 34...,78.0,Male
4,10610,spleen,HPA,3000,3000,0.4,4,478925 68 481909 87 484893 105 487863 154 4908...,21.0,Female


(1, 7)


,id,organ,data_source,img_height,img_width,pixel_size,tissue_thickness
0,10078,spleen,Hubmap,2023,2023,0.4945,4


In [3]:
def resize_tile_image_500_png(p_img, folder, size: int = 500):
    w = h = size
    im = Image.open(p_img)
    
    im_resize = im.resize((3000, 3000))
   
    im = np.array(im_resize)
    # https://stackoverflow.com/a/47581978/4521646
    tiles = [im[i:(i + h), j:(j + w), ...] for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    idxs = [(i, (i + h), j, (j + w)) for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    name, _ = os.path.splitext(os.path.basename(p_img))
    files = []
    for k, tile in enumerate(tiles):
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile = np.zeros_like(tiles[0])
            tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        p_img = os.path.join(folder, f"{name}_{k:03}.png")
        Image.fromarray(tile).save(p_img)
        files.append(p_img)
    return files, idxs


In [4]:
def resize_tile_image_500_jpg(p_img, folder, size: int = 500):
    w = h = size
    im = Image.open(p_img)
    
    im_resize = im.resize((3000, 3000))
   
    im = np.array(im_resize)
    # https://stackoverflow.com/a/47581978/4521646
    tiles = [im[i:(i + h), j:(j + w), ...] for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    idxs = [(i, (i + h), j, (j + w)) for i in range(0, im.shape[0], h) for j in range(0, im.shape[1], w)]
    name, _ = os.path.splitext(os.path.basename(p_img))
    files = []
    for k, tile in enumerate(tiles):
        if tile.shape[:2] != (h, w):
            tile_ = tile
            tile = np.zeros_like(tiles[0])
            tile[:tile_.shape[0], :tile_.shape[1], ...] = tile_
        p_img = os.path.join(folder, f"{name}_{k:03}.png")
        Image.fromarray(tile).save(p_img)
        files.append(p_img)
    return files, idxs

In [7]:
all_train_mclass_masks = glob(os.path.join("D:Dataset/dataset/hubmap-organ-segmentation/masked_images", '*.jpg'))

for p_mask in tqdm(sorted(all_train_mclass_masks)):
    tiles_seg, idxs = resize_tile_image_500_jpg(p_mask, 'D:Dataset/dataset/hubmap-organ-segmentation/masked_images_tiles')

  0%|          | 0/351 [00:00<?, ?it/s]

In [8]:
for p_img in tqdm(sorted(all_train_images)): 
    tiles_img, _  = resize_tile_image_500_jpg(p_img, "D:Dataset/dataset/hubmap-organ-segmentation/train_images_tiles/")

  0%|          | 0/351 [00:00<?, ?it/s]